# Test Code for the Galactic Plane Filter Metric

Code adapted from an example by Lynne Jones

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import rubin_sim.maf as maf

Load the baseline v2.0 as a test case OpSim:

In [2]:
from rubin_sim.data import get_baseline

opsim_fname = get_baseline()
print(opsim_fname)

runName = os.path.split(opsim_fname)[-1].replace('.db', '')
print(runName)
opsim_db = maf.OpsimDatabase(opsim_fname)

/Users/rstreet1/rubin_sim_data/sim_baseline/baseline_v2.0_10yrs.db
baseline_v2.0_10yrs


Taking an example sky location within the Galactic Bulge:

In [3]:
test_ra = (17.0 + 57.0/60.0 + 34.0/3600.0)*15.0
test_dec = (29.0 + 13.0/60.0 + 15.0/3600.0)*-1.0
test_slicer = maf.UserPointsSlicer(test_ra, test_dec)

In [73]:
import numpy as np
import healpy as hp
import rubin_sim.maf as maf
from astropy import units as u
from astropy_healpix import HEALPix
from astropy.coordinates import Galactic, TETE, SkyCoord

class galPlaneTimePerFilter(maf.BaseMetric):
    """Metric to evaluate for each HEALpix, the fraction of exposure time spent in each filter as
     a fraction of the total exposure time dedicated to that HEALpix.  The metric sums this over
     all HEALpix in the Galactic Plane/Magellanic Clouds region of interest and all filters and
     presents the result as a fraction of the value expected from the optimal survey strategy.

    Parameters
    ----------
    fieldRA : float, RA in degrees of a given pointing
    fieldDec : float, Dec in degrees of a given pointing
    filter : str, filter bandpass used for a given observation
    observationStartMJD : float, MJD timestamp of the start of a given observation
    visitExposureTime : float, exposure time in seconds
    """
    def __init__(self, cols=['fieldRA','fieldDec','filter',
                             'observationStartMJD','visitExposureTime','fiveSigmaDepth'],
                       metricName='galPlaneTimePerFilter',
                       **kwargs):
        """Kwargs must contain:
        filters  list Filterset over which to compute the metric
        """

        self.ra_col = 'fieldRA'
        self.dec_col = 'fieldDec'
        self.filterCol = 'filter'
        self.m5Col = 'fiveSigmaDepth'
        self.mjdCol = 'observationStartMJD'
        self.exptCol = 'visitExposureTime'
        self.filters = ['u','g', 'r', 'i', 'z', 'y']
        self.magCuts = {'u': 22.7, 'g': 24.1, 'r': 23.7, 'i': 23.1, 'z': 22.2, 'y': 21.4}
        cwd = os.getcwd()
        self.MAP_DIR = os.path.join(cwd,'../../data/galPlane_priority_maps')
        self.MAP_FILE_ROOT_NAME = 'GalPlane_priority_map'
        self.load_maps()
        self.calc_coaddmap()

        super().__init__(col=cols, metricName=metricName)

    def load_maps(self):
        self.NSIDE = 64
        self.NPIX = hp.nside2npix(self.NSIDE)
        for f in self.filters:
            setattr(self, 'map_'+str(f), hp.read_map(os.path.join(self.MAP_DIR,self.MAP_FILE_ROOT_NAME+'_'+str(f)+'.fits')))

    def calc_coaddmap(self):
        """For each HEALpix in the sky, sum the value of the priority weighting
        per filter.  This gives us the normalization factor to calculate the
        fractional proportional of exposure time"""
        
        coadded_map = np.zeros(self.map_i.shape)
        for i,f in enumerate(self.filters):
            coadded_map += getattr(self, 'map_'+f)
        
        self.coadded_map = coadded_map
        
    def calc_idealfExpT(self, filter_name, pixels):
        """Method to calculate the optimal value of the fExpT metric, in the event
        that a survey strategy perfectly overlaps the desired Galactic Plane footprint
        and the exposure time spent in the different filters at the desired relative
        proportions.  Calculation is made over all the pixels in a given dataSlice, 
        so that the metric can be accurately normalized later."""
        
        fexpt_per_pixel = np.zeros(len(pixels))
        fmap = getattr(self, 'map_'+filter_name)
        fexpt_per_pixel = fmap[pixels] / self.coadded_map[pixels]

        invalid = np.isnan(fexpt_per_pixel)
        fexpt_per_pixel[invalid] = 0.0

        # The value of the fExpT metric in ideal circumstance is the sum of all values:
        idealfExpT = fexpt_per_pixel.sum()
        
        return idealfExpT

    def run(self, dataSlice, slicePoint=None):

        # Identify those pixels that overlap the desired Galactic Plane survey region:
        # Returns list of pixel indices?
        #GP_overlap_pixels = np.where(self.map_i[pixels] > 0, 1, 0)
    
        # DOES A DATASLICE CONTAIN JUST A SUBSET OF PIXELS?
        total_expt_per_pixel = dataSlice[self.exptCol].sum()

        # Calculate the exposure time per filter per pixel as the fraction of the total
        # exposure time per pixel
        fexpt_per_filter = np.zeros(len(self.filters))
        for i,f in enumerate(self.filters):
            idx1 = np.where(dataSlice[self.filterCol] == f)[0]
            idx2 = np.where(dataSlice[self.m5Col] >= self.magCuts[f])[0]
            match = list(set(idx1).intersection(set(idx2)))

            coords_icrs = SkyCoord(dataSlice[self.ra_col][match], dataSlice[self.dec_col][match], frame='icrs', unit=(u.deg, u.deg))
            coords_gal = coords_icrs.transform_to(Galactic()) 
            ahp = HEALPix(nside=self.NSIDE, order='ring', frame=TETE())
            pixels = ahp.skycoord_to_healpix(coords_gal)
            
            idealfExpT = self.calc_idealfExpT(f,pixels)
            
            fexpt = dataSlice[self.exptCol][match].sum()/total_expt_per_pixel
            fexpt_per_filter[i] = fexpt / idealfExpT
        
        # Sum the fraction of exposure time over all pixels within the desired
        # survey region and filters:
        metric_value = fexpt_per_filter.sum()

        return metric_value


In [74]:
mymetric = galPlaneTimePerFilter()

Constraints, if any:

In [75]:
sqlconstraint = None

Construct the metric bundle from the metric, the slicer and the constraints, and add it to a bundle group:

In [76]:
bundle = maf.MetricBundle(mymetric, test_slicer, sqlconstraint, runName=runName)
g = maf.MetricBundleGroup({'test_metric': bundle}, opsim_db, outDir='test', resultsDb=None)

Calculate the metric:

In [77]:
g.runAll()

Querying database observations with no constraint for columns ['observationStartMJD', 'fiveSigmaDepth', 'fieldDec', 'filter', 'rotSkyPos', 'fieldRA', 'visitExposureTime'].
Found 2086980 visits
Running:  ['test_metric']
IDEAL exposure ratio per pixel, filter u :  [0.08284672 0.08284672 0.08284672 0.08284672 0.08284672 0.08284672
 0.08284672 0.08284672 0.03648649 0.08284672 0.08284672 0.08284672
 0.08284672 0.08284672 0.08284672 0.08284672 0.08284672 0.08284672
 0.08284672 0.08284672 0.08284672 0.08284672 0.08284672 0.08284672
 0.08284672 0.08284672 0.08284672 0.08284672 0.08284672 0.08284672
 0.08284672 0.08284672]
IDEAL exposure ratio sum, filter u :  2.604734661668968
IDEAL exposure ratio per pixel, filter g :  [0.20492701 0.20492701 0.20492701 0.20492701 0.20492701 0.20492701
 0.20492701 0.20492701 0.20492701 0.20492701 0.20492701 0.20492701
 0.20492701 0.20492701 0.20492701 0.20492701 0.20492701 0.20492701
 0.20492701 0.20492701 0.20492701 0.20492701 0.21824324 0.20492701
 0.2049270

In [79]:
bundle.metricValues

masked_array(data=[0.052741957418333804],
             mask=[False],
       fill_value=-666.0)